<a href="https://colab.research.google.com/github/BeemagoniVarun/Generative-AI_2025/blob/main/Gen_ai_p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd


df = pd.read_csv('/content/kaggle_poem_dataset.csv')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load pre-trained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Set the device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def generate_poem(prompt, max_length=100):
    inputs = tokenizer.encode_plus(
        prompt,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length)
    poem = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return poem

# Generate poems using prompts from the dataset
for index, row in df.iterrows():
    prompt = row['Content']
    generated_poem = generate_poem(prompt)
    print(f"Prompt: {prompt}\nGenerated Poem:\n{generated_poem}\n")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Streaming output truncated to the last 5000 lines.
the dog turning. the dog turning. the mattress, the dog turning. the windmill. nothing alive—nothing alive—in Dakota. I saw that, and nothing alive—and nothing alive—in Dakota. In Dakota, and nothing alive—in Dakota. In Dakota, and nothing alive—in Dakota. In Dakota, and nothing alive—in Dakota. In Dakota, and nothing alive—in Dakota. In Dakota, and nothing alive—in Dakota

Prompt: He was a big man, says the size of his shoes
on a pile of broken dishes by the house;
a tall man too, says the length of the bed
in an upstairs room; and a good, God-fearing man,
says the Bible with a broken back
on the floor below the window, dusty with sun;
but not a man for farming, say the fields
cluttered with boulders and the leaky barn.

A woman lived with him, says the bedroom wall
papered with lilacs and the kitchen shelves
covered with oilcloth, and they had a child,
says the sandbox made from a tractor tire.
Money was scarce, say the jars of plum 

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from torch.utils.data import Dataset, DataLoader

# Custom dataset class for our data
class PoemDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        prompt = self.df.iloc[idx]['Content']
        poem = self.df.iloc[idx]['TargetPoem']  # Assuming you have target poems

        inputs = self.tokenizer.encode_plus(
            prompt,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        labels = self.tokenizer.encode_plus(
            poem,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels['input_ids'].squeeze()
        }

# Load pre-trained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Set the device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Create dataset and data loader
dataset = PoemDataset(df, tokenizer)
batch_size = 16
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Fine-tune the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagate the loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')

# Generate poems using the fine-tuned model
def generate_poem(prompt, max_length=100):
    inputs = tokenizer.encode_plus(
        prompt,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length)
    poem = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return poem

# Generate poems using prompts from the dataset
for index, row in df.iterrows():
    prompt = row['prompt']
    generated_poem = generate_poem(prompt)
    print(f"Prompt: {prompt}\nGenerated Poem:\n{generated_poem}\n")
